In [1]:
from pyspark.sql import SparkSession
allocated_memory = 18 * 0.75 

# create a SparkSession
spark = SparkSession.builder.appName("ReadJSON")\
.config("spark.executor.memory", "6g") \
.master("local[*]")  \
.config("spark.driver.memory", "4g") \
.config("spark.network.timeout", "800s")\
.config("spark.executor.heartbeatInterval", "200s")\
.config("spark.executor.extraJavaOptions", "-XX:+UseG1GC")\
.config("spark.driver.extraJavaOptions", "-XX:+UseG1GC")\
.config("spark.memory.fraction", "0.8") \
.getOrCreate()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/26 20:41:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/26 20:41:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 54934)
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
  File "/opt/homebrew/Cellar/python@3.12/3.12.2_1/Frameworks/Python.framework/Versions/3.12/lib/python3.12/socketserver.py", line 761, in __init__
    self.handle()
  File "/Users/kravisankaran/trial/lib/python3.12/site-pa

In [4]:
print('Default Parallelism :', spark.sparkContext.defaultParallelism)

Default Parallelism : 12


In [6]:
# Check the default partition size
partition_size = spark.conf.get("spark.sql.files.maxPartitionBytes").replace("b","")
print(f"Partition Size: {partition_size} in bytes and {int(partition_size) / 1024 / 1024} in MB")

Partition Size: 134217728 in bytes and 128.0 in MB


In [8]:
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType
#"reviewerID": "A8WEXFRWX1ZHH", 
# "asin": "0209688726", 
# "style": {"Color:": " AC"}, 
# "reviewerName": "Goldengate",
# Define the schema
schema = StructType([
    StructField("overall", FloatType(), True),
    StructField("verified", BooleanType(), True),
    StructField("reviewTime", StringType(), True),
    StructField("reviewerID", StringType(), True),
    StructField("asin", StringType(), True),
    StructField("style", StructType([StructField("Color:", StringType(), True)]), True),
    StructField("reviewerName", StringType(), True),
    StructField("reviewText", StringType(), True),
    StructField("unixReviewTime", IntegerType(), True)
    
])


json_df = spark.read.schema(schema).json("combined_train_data_chunked_10mb_latest.json")
json_test_df = spark.read.schema(schema).json("combined_test_data_chunked_10mb_latest.json")
json_df.show(5)
json_test_df.show(5)

# 

+-------+--------+-----------+--------------+----------+------+---------------+--------------------+--------------+
|overall|verified| reviewTime|    reviewerID|      asin| style|   reviewerName|          reviewText|unixReviewTime|
+-------+--------+-----------+--------------+----------+------+---------------+--------------------+--------------+
|    2.0|    true| 01 1, 2016|A22DGMXU20E35S|B001CI6AYO|{NULL}|         Angela|Bought this for m...|    1451606400|
|    5.0|    true|12 11, 2017|A1LVJV4ZU0IXSM|B000C5UFGW|  NULL|   Carl Colberg|Fits 89 22RE, wit...|    1512950400|
|    4.0|    true|12 11, 2007|  ACWOXD1LOTIN|B000H8RV6Q|{NULL}|Aaron I. Strang|I bought this cd ...|    1197331200|
|    5.0|    true|05 24, 2016|A1SFZH5JWWII21|B00DL7QCHY|{NULL}|         Sean L|Very useful tool,...|    1464048000|
|    5.0|    true|07 28, 2016|A1PLE0QLRO7CB7|B017RYDHG0|  NULL|Kindle Customer|"Good trainer for...|    1469664000|
+-------+--------+-----------+--------------+----------+------+---------

In [9]:
# Print the number of entries in the dataframe
print("Number of entries in the dataframe before pre processing: ", json_df.count())
print("Number of entries in the dataframe before pre processing: ", json_test_df.count())

Number of entries in the dataframe before pre processing:  11321389


Number of entries in the dataframe before pre processing:  2832499


In [11]:
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, FloatType, BooleanType, IntegerType, ArrayType
# targetUDF = F.udf(lambda x: 1 if x >= 4.0 else (0 if x == 3.0 else -1), IntegerType())
targetUDF = F.udf(lambda x: 1 if x >= 4.0 else 0, IntegerType())
import re
import nltk
from nltk.corpus import stopwords

In [12]:
reduced_df = json_df.select("overall", "reviewerID", "asin", "reviewText")
reduced_test_df = json_test_df.select("overall", "reviewerID", "asin", "reviewText")



In [13]:
reduced_df.show(5)
reduced_test_df.show(5)

+-------+--------------+----------+--------------------+
|overall|    reviewerID|      asin|          reviewText|
+-------+--------------+----------+--------------------+
|    2.0|A22DGMXU20E35S|B001CI6AYO|Bought this for m...|
|    5.0|A1LVJV4ZU0IXSM|B000C5UFGW|Fits 89 22RE, wit...|
|    4.0|  ACWOXD1LOTIN|B000H8RV6Q|I bought this cd ...|
|    5.0|A1SFZH5JWWII21|B00DL7QCHY|Very useful tool,...|
|    5.0|A1PLE0QLRO7CB7|B017RYDHG0|"Good trainer for...|
+-------+--------------+----------+--------------------+
only showing top 5 rows

+-------+--------------+----------+--------------------+
|overall|    reviewerID|      asin|          reviewText|
+-------+--------------+----------+--------------------+
|    5.0|A2LSCFZM2FBZK7|0486427706|The stained glass...|
|    5.0|A3IXP5VS847GE5|0486427706|My 11 y.o. loved ...|
|    5.0|A2HK5AVQW6AUQ5|0486427706|             love it|
|    5.0|A18MVTKTTE8OS8|0486448789|Sometimes you nee...|
|    5.0|A2C2TLRMMMLAJV|0486448789|These little book...|
+-----

In [14]:
unique_df = reduced_df.dropDuplicates(["reviewerID", "asin"])
print("Number of training entries in the dataframe after removing duplicates: ", unique_df.count())

unique_test_df = reduced_test_df.dropDuplicates(["reviewerID", "asin"])
print("Number of testing entries in the dataframe after removing duplicates: ", unique_test_df.count())

Number of training entries in the dataframe after removing duplicates:  10936206


Number of testing entries in the dataframe after removing duplicates:  2806599


In [15]:
def preProcess(text):
    # Should return a list of tokens
    text = re.sub(r"(\w)([.,;:!?'\"”\)])", r"\1 \2", text)
    text = re.sub(r"([.,;:!?'\"“\(])(\w)", r"\1 \2", text)    
    text = text.lower()
    tokens = word_tokenize(text)    
    return tokens

In [16]:
df_sentiment = unique_df.withColumn("sentiment", targetUDF(unique_df["overall"]))
df_test_sentiment = unique_test_df.withColumn("sentiment", targetUDF(unique_test_df["overall"]))

In [17]:
df_sentiment.show(5)
df_test_sentiment.show(5)

+-------+--------------------+----------+--------------------+---------+
|overall|          reviewerID|      asin|          reviewText|sentiment|
+-------+--------------------+----------+--------------------+---------+
|    5.0|A0015332H21AK8WZ0ZCS|B005G030TC|These collars are...|        1|
|    4.0| A0020356UF96ZV361ST|B00ZI5OVFM|It is a love stor...|        1|
|    2.0| A0020356UF96ZV361ST|B015X7KEDM|This book is not ...|        0|
|    5.0|A0024936S1WI02OHH9DP|B016AG5DR2|Looks great fits ...|        1|
|    5.0|A0034986DWR7WEDQN0GV|B001VJZO2S|           excellent|        1|
+-------+--------------------+----------+--------------------+---------+
only showing top 5 rows



+-------+-------------------+----------+--------------------+---------+
|overall|         reviewerID|      asin|          reviewText|sentiment|
+-------+-------------------+----------+--------------------+---------+
|    3.0|A0020356UF96ZV361ST|B00FDXFFW2|I guess you can s...|        0|
|    4.0|A0020356UF96ZV361ST|B00H6VZ0SS|This girl has bee...|        1|
|    4.0|A0020356UF96ZV361ST|B00XQOGWV8|Mario is a sorry ...|        1|
|    4.0|A0020356UF96ZV361ST|B014HD23EQ|This father had h...|        1|
|    4.0|A0020356UF96ZV361ST|B018RSH2FW|This guy is and h...|        1|
+-------+-------------------+----------+--------------------+---------+
only showing top 5 rows



In [18]:
from pyspark.ml.feature import Tokenizer

# use PySparks build in tokenizer to tokenize tweets
tokenizer = Tokenizer(inputCol  = "reviewText",
                      outputCol = "token")
# Remove the rows with missing values and tokenize
df_train_tokenized = tokenizer.transform(df_sentiment.filter(unique_df.reviewText.isNotNull()))
df_test_tokenized = tokenizer.transform(df_test_sentiment.filter(unique_test_df.reviewText.isNotNull()))

In [19]:

df_train_tokenized.show(5)
df_test_tokenized.show(5)

+-------+--------------------+----------+--------------------+---------+--------------------+
|overall|          reviewerID|      asin|          reviewText|sentiment|               token|
+-------+--------------------+----------+--------------------+---------+--------------------+
|    5.0|A0015332H21AK8WZ0ZCS|B005G030TC|These collars are...|        1|[these, collars, ...|
|    4.0| A0020356UF96ZV361ST|B00ZI5OVFM|It is a love stor...|        1|[it, is, a, love,...|
|    2.0| A0020356UF96ZV361ST|B015X7KEDM|This book is not ...|        0|[this, book, is, ...|
|    5.0|A0024936S1WI02OHH9DP|B016AG5DR2|Looks great fits ...|        1|[looks, great, fi...|
|    5.0|A0034986DWR7WEDQN0GV|B001VJZO2S|           excellent|        1|         [excellent]|
+-------+--------------------+----------+--------------------+---------+--------------------+
only showing top 5 rows



+-------+-------------------+----------+--------------------+---------+--------------------+
|overall|         reviewerID|      asin|          reviewText|sentiment|               token|
+-------+-------------------+----------+--------------------+---------+--------------------+
|    3.0|A0020356UF96ZV361ST|B00FDXFFW2|I guess you can s...|        0|[i, guess, you, c...|
|    4.0|A0020356UF96ZV361ST|B00H6VZ0SS|This girl has bee...|        1|[this, girl, has,...|
|    4.0|A0020356UF96ZV361ST|B00XQOGWV8|Mario is a sorry ...|        1|[mario, is, a, so...|
|    4.0|A0020356UF96ZV361ST|B014HD23EQ|This father had h...|        1|[this, father, ha...|
|    4.0|A0020356UF96ZV361ST|B018RSH2FW|This guy is and h...|        1|[this, guy, is, a...|
+-------+-------------------+----------+--------------------+---------+--------------------+
only showing top 5 rows



In [21]:
import re

def removeRegex(tokens: list) -> list:
    """
    Removes hashtags, call outs and web addresses from tokens.
    """
    # Use a raw string for regular expressions to avoid escape sequence warnings
    expr = r'(@[A-Za-z0-9_]+)|(#[A-Za-z0-9_]+)|'+\
           r'(https?://[^\s<>"]+|www\.[^\s<>"]+)'
    regex = re.compile(expr)
    cleaned = [t for t in tokens if not regex.search(t) and len(t) > 0]

    return cleaned


In [22]:
removeWEBUDF = F.udf(removeRegex, ArrayType(StringType()))

In [23]:
def normalize(tokens : list) -> list:
    """
    Removes non-english characters and returns lower case versions of words.
    """
    subbed   = [re.sub("[^a-zA-Z]+", "", s).lower() for s in tokens]
    
    filtered = filter(None, subbed)
    
    return list(filtered)


normalizeUDF = F.udf(normalize, ArrayType(StringType()))

In [24]:
# remove hashtags, call outs and web addresses
df4_train = df_train_tokenized.withColumn("tokens_re", removeWEBUDF(df_train_tokenized["token"]))
df4_test = df_test_tokenized.withColumn("tokens_re", removeWEBUDF(df_test_tokenized["token"]))
# remove non english characters
df4_train = df4_train.withColumn("tokens_clean", normalizeUDF(df4_train["tokens_re"]))
df4_test = df4_test.withColumn("tokens_clean", normalizeUDF(df4_test["tokens_re"]))

# rename columns
df5_train = df4_train.drop("token","tokens_re")
df5_test = df4_test.drop("token","tokens_re")
df5_train = df5_train.withColumnRenamed("tokens_clean", "tokens")
df5_test = df5_test.withColumnRenamed("tokens_clean", "tokens")

# remove reviews where the tokens array is empty, i.e. where it was just
# a hashtag, callout, numbers, web adress etc.
df6_train = df5_train.where(F.size(F.col("tokens")) > 0)
df6_test = df5_test.where(F.size(F.col("tokens")) > 0)

In [25]:
df_train_for_model = df6_train.select("reviewText","sentiment")\
        .withColumnRenamed("sentiment", "label")
df_test_for_model = df6_test.select("reviewText","sentiment").withColumnRenamed("sentiment", "label")

In [26]:
from pyspark.sql.functions import rand

# Assuming 'df' is your DataFrame
shuffled_train_df = df_train_for_model.orderBy(rand())
shuffled_test_df = df_test_for_model.orderBy(rand())

# Show the shuffled DataFrame
shuffled_train_df.show(10)
shuffled_test_df.show(10)

+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|just be careful i...|    1|
|good size toy and...|    1|
|If there was ever...|    1|
|It is such a good...|    1|
|I add a tsp. of t...|    1|
|Converted my Port...|    1|
|Not what i expected.|    1|
|Cute toy for youn...|    1|
|Amazing! Not one ...|    1|
|    It's basic cart.|    0|
+--------------------+-----+
only showing top 10 rows



+--------------------+-----+
|          reviewText|label|
+--------------------+-----+
|Switchback LED li...|    1|
|This worked very ...|    1|
|I was gifted this...|    1|
|Excellent, i was ...|    1|
|      Great product!|    1|
|This is one Quali...|    1|
|Nice variety, and...|    1|
|  Good but not Great|    1|
|I wish they were ...|    1|
|Loved it . Happy ...|    1|
+--------------------+-----+
only showing top 10 rows



In [ ]:
# # Printing label distribution for training and testing data
# print("Training data label distribution")
# shuffled_train_df.groupBy("label").count().show()

# print("Testing data label distribution")
# shuffled_test_df.groupBy("label").count().show()

In [32]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator


evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")

# get the name of the metric used
evaluator.getMetricName()

'areaUnderROC'

In [ ]:
# import pickle
# with open("model1.pkl", "wb") as f:
#     pickle.dump(model1, f)

In [33]:
# create tokens from reviews
tk = Tokenizer(inputCol= "reviewText", outputCol = "tokens")

# create term frequencies for each of the tokens
tf1 = HashingTF(inputCol="tokens", outputCol="rawFeatures", numFeatures=1e5)

# create tf-idf for each of the tokens
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=2.0)

# create basic logistic regression model
lr = LogisticRegression(maxIter=20)

In [35]:
from pyspark.ml.feature import StopWordsRemover
sw  = StopWordsRemover(inputCol="tokens", outputCol="filtered")
tf2 = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1e5)

In [ ]:
# shuffled_train_df_partition = shuffled_train_df.repartition(5)
# shuffled_test_df_partition = shuffled_test_df.repartition(5)

In [ ]:
# print(shuffled_train_df_partition.rdd.getNumPartitions())
# print(shuffled_test_df_partition.rdd.getNumPartitions())

In [28]:
from nltk.stem.porter import PorterStemmer

In [29]:
from pyspark import keyword_only
import numpy as np
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param


class PorterStemming(Transformer, HasInputCol, HasOutputCol):
    """
    PosterStemming class using the NLTK Porter Stemmer
    
    This comes from https://stackoverflow.com/questions/32331848/create-a-custom-transformer-in-pyspark-ml
    Adapted to work with the Porter Stemmer from NLTK.
    """
    
    @keyword_only
    def __init__(self, 
                 inputCol  : str = None, 
                 outputCol : str = None, 
                 min_size  : int = None):
        """
        Constructor takes in the input column name, output column name,
        plus the minimum legnth of a token (min_size)
        """
        # call Transformer classes constructor since were extending it.
        super(Transformer, self).__init__()

        # set Parameter objects minimum token size
        self.min_size = Param(self, "min_size", "")
        self._setDefault(min_size=0)

        # set the input keywork arguments
        kwargs = self._input_kwargs
        self.setParams(**kwargs)

        # initialize Stemmer object
        self.stemmer  = PorterStemmer()

        
    @keyword_only
    def setParams(self, 
                  inputCol  : str = None, 
                  outputCol : str = None, 
                  min_size  : int = None
      ) -> None:
        """
        Function to set the keyword arguemnts
        """
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    

    def _stem_func(self, words  : list) -> list:
        """
        Stemmer function call that performs stemming on a
        list of tokens in words and returns a list of tokens
        that have meet the minimum length requiremnt.
        """
        # We need a way to get min_size and cannot access it 
        # with self.min_size
        min_size       = self.getMinSize()

        # stem that actual tokens by applying 
        # self.stemmer.stem function to each token in 
        # the words list
        stemmed_words  = map(self.stemmer.stem, words)

        # now create the new list of tokens from
        # stemmed_words by filtering out those
        # that are not of legnth > min_size
        filtered_words = filter(lambda x: len(x) > min_size, stemmed_words)

        return list(filtered_words)
    
    def _transform(self, df: DataFrame) -> DataFrame:
        """
        Transform function is the method that is called in the 
        MLPipleline.  We have to override this function for our own use
        and have it call the _stem_func.

        Notice how it takes in a type DataFrame and returns type Dataframe
        """
        # Get the names of the input and output columns to use
        out_col       = self.getOutputCol()
        in_col        = self.getInputCol()

        # create the stemming function UDF by wrapping the stemmer 
        # method function
        stem_func_udf = F.udf(self._stem_func, ArrayType(StringType()))
        
        # now apply that UDF to the column in the dataframe to return
        # a new column that has the same list of words after being stemmed
        df2           = df.withColumn(out_col, stem_func_udf(df[in_col]))

        return df2
  
  
    def setMinSize(self,value):
        """
        This method sets the minimum size value
        for the _paramMap dictionary.
        """
        self._paramMap[self.min_size] = value
        return self

    def getMinSize(self) -> int:
        """
        This method uses the parent classes (Transformer)
        .getOrDefault method to get the minimum
        size of a token.
        """
        return self.getOrDefault(self.min_size)


In [36]:
from pyspark.ml.feature import NGram
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
bigram2 = NGram(inputCol="stemmed", outputCol="bigrams", n=2)

tf6     = HashingTF(inputCol="bigrams", outputCol="rawFeatures", numFeatures=2e5)

idf     = IDF(inputCol="rawFeatures", outputCol="features")

lr      = LogisticRegression(maxIter=10)

stem_bigram_pipeline  = Pipeline(stages= [bigram2, sw, tf6, idf, lr])

paramGrid = ParamGridBuilder() \
                        .addGrid(idf.minDocFreq, [2, 5]) \
                        .addGrid(lr.regParam, [0.0, 0.1]) \
                        .build()
crossval = CrossValidator(estimator          = stem_bigram_pipeline,
                          estimatorParamMaps = paramGrid,
                          evaluator          = BinaryClassificationEvaluator(),
                          numFolds           = 2,
                          parallelism= 2
                          )


# model    = crossval.fit(train_stem)
# predictions   = model.transform(test_stem)
# score         = evaluator.evaluate(predictions)
# print("AUC SCORE: {}".format(score))

In [37]:
from pyspark.sql.functions import monotonically_increasing_id, row_number
import pyspark.sql.functions as F
from pyspark.sql.window import Window
shuffled_train_df = shuffled_train_df.withColumn('original_order', monotonically_increasing_id())
shuffled_train_df = shuffled_train_df.withColumn('row_num', row_number().over(Window.orderBy('original_order')))
shuffled_train_df = shuffled_train_df.drop('original_order')

shuffled_train_df.show(2)
shuffled_test_df = shuffled_test_df.withColumn('original_order', monotonically_increasing_id())
shuffled_test_df = shuffled_test_df.withColumn('row_num', row_number().over(Window.orderBy('original_order')))
shuffled_test_df = shuffled_test_df.drop('original_order')
shuffled_test_df.show(2)

shuffled_train_df.cache()
shuffled_test_df.cache()

24/04/26 20:55:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 20:55:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 20:55:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 20:56:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 20:56:07 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:00:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 2

+--------------------+-----+-------+
|          reviewText|label|row_num|
+--------------------+-----+-------+
|just be careful i...|    1|      1|
|good size toy and...|    1|      2|
+--------------------+-----+-------+
only showing top 2 rows



24/04/26 21:00:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:00:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:00:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:00:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:00:34 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:01:39 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 2

+--------------------+-----+-------+
|          reviewText|label|row_num|
+--------------------+-----+-------+
|Switchback LED li...|    1|      1|
|This worked very ...|    1|      2|
+--------------------+-----+-------+
only showing top 2 rows



24/04/26 21:01:40 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:01:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:01:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:01:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:01:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:01:41 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


DataFrame[reviewText: string, label: int, row_num: int]

In [39]:
from pyspark.sql.functions import col
stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")
train_shuffle_small = shuffled_train_df.filter( (col('row_num') >= 1) & (col('row_num') <=50000))
#stem_pipeline = Pipeline(stages= [tk, stem2]).fit(shuffled_train_df_partition)
stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train_shuffle_small)

#train_stem = stem_pipeline.transform(shuffled_train_df_partition)\
                          #.where(F.size(F.col("stemmed")) >= 1)
train_stem = stem_pipeline.transform(train_shuffle_small)\
                          .where(F.size(F.col("stemmed")) >= 1)


# test_stem  = stem_pipeline.transform(shuffled_test_df_partition)\
#                           .where(F.size(F.col("stemmed")) >= 1)
test_stem  = stem_pipeline.transform(shuffled_test_df)\
                          .where(F.size(F.col("stemmed")) >= 1)

# cache them to avoid running stemming 
# each iteration in the grid search
train_stem.cache()
test_stem.cache()

24/04/26 21:04:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:04:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:04:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:04:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


DataFrame[reviewText: string, label: int, row_num: int, tokens: array<string>, stemmed: array<string>]

In [ ]:
from pyspark.ml.feature import NGram
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
bigram2 = NGram(inputCol="stemmed", outputCol="bigrams", n=2)

tf6     = HashingTF(inputCol="bigrams", outputCol="rawFeatures", numFeatures=2e5)

idf     = IDF(inputCol="rawFeatures", outputCol="features")

lr      = LogisticRegression(maxIter=10)

stem_bigram_pipeline  = Pipeline(stages= [bigram2, tf6, idf, lr])

paramGrid = ParamGridBuilder() \
                        .addGrid(idf.minDocFreq, [2, 5]) \
                        .addGrid(lr.regParam, [0.0, 0.1]) \
                        .build()
crossval = CrossValidator(estimator          = stem_bigram_pipeline,
                          estimatorParamMaps = paramGrid,
                          evaluator          = BinaryClassificationEvaluator(),
                          numFolds           = 2,
                          parallelism= 2
                          )


model    = crossval.fit(train_stem)
predictions   = model.transform(test_stem)
score         = evaluator.evaluate(predictions)
print("AUC SCORE: {}".format(score))

In [ ]:
model.save("cross_val_model_firstRun")

In [ ]:
# train_shuffle_small = train_shuffle_small.filter( (col('row_num') >= 340001) & (col('row_num') <=400000))
rows = shuffled_train_df.count()
print(rows)

In [ ]:
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.pipeline import PipelineModel
i = 50001
# 10927431
file_name_first = "cross_val_model_firstRun"
file_name_string = ""
file_name = 1

for i in range(50001, 3480000, 70000):
    # Add your indented block of code here
    if i == 50001:
        model_path = "cross_val_model_firstRun"
        model = CrossValidatorModel.load(model_path)
        stages_steps = model.bestModel.stages
    else:
        model_path = file_name_string
        model = PipelineModel.load(model_path)
        stages_steps = model.stages
    train_shuffle_small = shuffled_train_df.filter( (col('row_num') >= i) & (col('row_num') <=i+70000))
    stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")
    stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train_shuffle_small)
    updated_model = Pipeline(stages = stages_steps).fit(train_stem)
    predictions = updated_model.transform(test_stem)
    score         = evaluator.evaluate(predictions)
    print("AUC SCORE in this run : {}".format(score))
    file_name_string = "cross_val_model_" + str(file_name + 1) + "_Runs"
    updated_model.save(file_name_string)
    print("Model saved with name : ", file_name_string)
    file_name += 1
    i += 70000
    
    
    


In [ ]:
from pyspark.ml.pipeline import PipelineModel
model = PipelineModel.load("/Users/kravisankaran/Desktop/big-data-final-project/cross_val_model_49_Runs")

In [29]:
# from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.pipeline import PipelineModel
i = 3480001
# 10927431

file_name_string = ""
file_name = 49
for i in range(3480001, 7000000, 90000):
    # Add your indented block of code here
    if i == 3480001:
        print("Run number : ", file_name)
        model_path = "/Users/kravisankaran/Desktop/big-data-final-project/cross_val_model_49_Runs"
    else:
        print("Run number : ", file_name)
        model_path = file_name_string
    model = PipelineModel.load(model_path)
    stages_steps = model.stages
    train_shuffle_small = shuffled_train_df.filter( (col('row_num') >= i) & (col('row_num') <=i+90000))
    stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")
    stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train_shuffle_small)
    updated_model = Pipeline(stages = stages_steps).fit(train_stem)
    predictions = updated_model.transform(test_stem)
    score         = evaluator.evaluate(predictions)
    print("AUC SCORE in this run : {}".format(score))
    file_name_string = "cross_val_model_" + str(file_name + 1) + "_Runs"
    updated_model.save(file_name_string)
    print("Model saved with name : ", file_name_string)
    file_name += 1
    i += 90000
    
    
    


Run number :  49


24/04/23 13:09:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 13:09:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 13:10:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 13:10:05 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 13:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 13:10:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 1

AUC SCORE in this run : 0.8242746099293129


24/04/23 13:30:20 WARN TaskSetManager: Stage 110 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 13:30:21 WARN TaskSetManager: Stage 114 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_50_Runs
Run number :  50


...

24/04/23 16:50:27 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/23 16:50:31 WARN MemoryStore: Not enough space to cache rdd_263_0 in memory! (computed 2.5 GiB so far)
24/04/23 16:50:33 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMa

AUC SCORE in this run : 0.8242746099293129


24/04/23 16:55:13 WARN TaskSetManager: Stage 1787 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_89_Runs


24/04/23 16:55:13 WARN TaskSetManager: Stage 1791 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


In [41]:
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.pipeline import PipelineModel
i = 7000001
# 10927431

file_name_string = ""
file_name = 89
for i in range(7000001, 8260000, 90000):
    # Add your indented block of code here
    if i == 7000001:
        print("Run number : ", file_name)
        model_path = "/Users/kravisankaran/Desktop/big-data-final-project/cross_val_model_89_Runs"
    else:
        print("Run number : ", file_name)
        model_path = file_name_string
    model = PipelineModel.load(model_path)
    stages_steps = model.stages
    train_shuffle_small = shuffled_train_df.filter( (col('row_num') >= i) & (col('row_num') <=i+90000))
    stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")
    stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train_shuffle_small)
    updated_model = Pipeline(stages = stages_steps).fit(train_stem)
    predictions = updated_model.transform(test_stem)
    score         = evaluator.evaluate(predictions)
    print("AUC SCORE in this run : {}".format(score))
    file_name_string = "cross_val_model_" + str(file_name + 1) + "_Runs"
    updated_model.save(file_name_string)
    print("Model saved with name : ", file_name_string)
    file_name += 1
    i += 90000

Run number :  89


24/04/23 23:23:47 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 23:23:48 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 23:25:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 23:25:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 23:25:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 23:25:56 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/23 2

AUC SCORE in this run : 0.8242746099293129


24/04/23 23:46:35 WARN TaskSetManager: Stage 311 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/23 23:46:36 WARN TaskSetManager: Stage 315 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_90_Runs
Run number :  90


...

24/04/24 07:08:37 WARN TaskSetManager: Stage 913 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/24 07:08:38 WARN TaskSetManager: Stage 917 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_104_Runs
Run number :  104


24/04/24 07:09:32 WARN TransportChannelHandler: Exception in connection from albertastein-2.lan/192.168.86.26:61560
java.io.IOException: Operation timed out
	at sun.nio.ch.FileDispatcherImpl.read0(Native Method)
	at sun.nio.ch.SocketDispatcher.read(SocketDispatcher.java:39)
	at sun.nio.ch.IOUtil.readIntoNativeBuffer(IOUtil.java:223)
	at sun.nio.ch.IOUtil.read(IOUtil.java:192)
	at sun.nio.ch.SocketChannelImpl.read(SocketChannelImpl.java:379)
	at io.netty.buffer.PooledByteBuf.setBytes(PooledByteBuf.java:254)
	at io.netty.buffer.AbstractByteBuf.writeBytes(AbstractByteBuf.java:1132)
	at io.netty.channel.socket.nio.NioSocketChannel.doReadBytes(NioSocketChannel.java:357)
	at io.netty.channel.nio.AbstractNioByteChannel$NioByteUnsafe.read(AbstractNioByteChannel.java:151)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey(NioEventLoop.java:788)
	at io.netty.channel.nio.NioEventLoop.processSelectedKeysOptimized(NioEventLoop.java:724)
	at io.netty.channel.nio.NioEventLoop.processSelectedKey

Py4JJavaError: An error occurred while calling o7338.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 929.0 failed 1 times, most recent failure: Lost task 0.0 in stage 929.0 (TID 1430) (albertastein-2.lan executor driver): TaskResultLost (result lost from block manager)
Driver stacktrace:
	at org.apache.spark.scheduler.DAGScheduler.failJobAndIndependentStages(DAGScheduler.scala:2856)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2(DAGScheduler.scala:2792)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$abortStage$2$adapted(DAGScheduler.scala:2791)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at org.apache.spark.scheduler.DAGScheduler.abortStage(DAGScheduler.scala:2791)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$handleTaskSetFailed$1(

24/04/24 07:09:58 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:123)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.isExecutorAlive$lzycompute$1(BlockManagerMasterEndpoint.scala:688)
	at org.apache.spark.storage.BlockManagerMasterE

In [40]:
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.pipeline import PipelineModel
i = 8260001
# 10927431

file_name_string = ""
file_name = 105
for i in range(8260001, 9260000, 90000):
    # Add your indented block of code here
    if i == 8260001:
        print("Run number : ", file_name)
        model_path = "/Users/kravisankaran/Desktop/big-data-final-project/cross_val_model_104_Runs"
    else:
        print("Run number : ", file_name)
        model_path = file_name_string
    model = PipelineModel.load(model_path)
    stages_steps = model.stages
    train_shuffle_small = shuffled_train_df.filter( (col('row_num') >= i) & (col('row_num') <=i+90000))
    stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")
    stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train_shuffle_small)
    train_stem = stem_pipeline.transform(train_shuffle_small)\
                          .where(F.size(F.col("stemmed")) >= 1)
    updated_model = Pipeline(stages = stages_steps).fit(train_stem)
    predictions = updated_model.transform(test_stem)
    score         = evaluator.evaluate(predictions)
    print("AUC SCORE in this run : {}".format(score))
    file_name_string = "cross_val_model_" + str(file_name + 1) + "_Runs"
    updated_model.save(file_name_string)
    print("Model saved with name : ", file_name_string)
    file_name += 1
    i += 90000

Run number :  105


24/04/26 21:05:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:05:19 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:06:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:06:23 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:06:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 21:06:26 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/04/26 2

AUC SCORE in this run : 0.8242746099293129


24/04/26 21:26:51 WARN TaskSetManager: Stage 124 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/26 21:26:53 WARN TaskSetManager: Stage 128 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_106_Runs
Run number :  106


...

Model saved with name :  cross_val_model_116_Runs
Run number :  116


24/04/26 22:13:21 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/26 22:13:25 WARN MemoryStore: Not enough space to cache rdd_296_0 in memory! (computed 2.5 GiB so far)


AUC SCORE in this run : 0.8242746099293129


24/04/26 22:17:54 WARN TaskSetManager: Stage 597 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/26 22:17:55 WARN TaskSetManager: Stage 601 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_117_Runs


In [41]:
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.pipeline import PipelineModel
i = 9260001
# 10927431

file_name_string = ""
file_name = 117
for i in range(9260001, 10260001, 90000):
    # Add your indented block of code here
    if i == 9260001:
        print("Run number : ", file_name)
        model_path = "/Users/kravisankaran/Desktop/big-data-final-project/cross_val_model_117_Runs"
    else:
        print("Run number : ", file_name)
        model_path = file_name_string
    model = PipelineModel.load(model_path)
    stages_steps = model.stages
    train_shuffle_small = shuffled_train_df.filter( (col('row_num') >= i) & (col('row_num') <=i+90000))
    stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")
    stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train_shuffle_small)
    train_stem = stem_pipeline.transform(train_shuffle_small)\
                          .where(F.size(F.col("stemmed")) >= 1)
    updated_model = Pipeline(stages = stages_steps).fit(train_stem)
    predictions = updated_model.transform(test_stem)
    score         = evaluator.evaluate(predictions)
    print("AUC SCORE in this run : {}".format(score))
    file_name_string = "cross_val_model_" + str(file_name + 1) + "_Runs"
    updated_model.save(file_name_string)
    print("Model saved with name : ", file_name_string)
    file_name += 1
    i += 90000

Run number :  117


24/04/26 22:49:28 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/26 22:49:33 WARN MemoryStore: Not enough space to cache rdd_296_0 in memory! (computed 2.5 GiB so far)


AUC SCORE in this run : 0.8242746099293129


24/04/26 22:54:08 WARN TaskSetManager: Stage 640 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/26 22:54:09 WARN TaskSetManager: Stage 644 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_118_Runs
Run number :  118


...

Model saved with name :  cross_val_model_128_Runs
Run number :  128


24/04/26 23:39:35 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/26 23:39:39 WARN MemoryStore: Not enough space to cache rdd_296_0 in memory! (computed 2.5 GiB so far)


AUC SCORE in this run : 0.8242746099293129


24/04/26 23:44:08 WARN TaskSetManager: Stage 1113 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/26 23:44:09 WARN TaskSetManager: Stage 1117 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_129_Runs


In [43]:
from pyspark.ml.tuning import CrossValidatorModel
from pyspark.ml.pipeline import PipelineModel
i = 10260002
# 10927431

file_name_string = ""
file_name = 129
for i in range(10260002, 10927431, 90000):
    # Add your indented block of code here
    if i == 10260002:
        print("Run number : ", file_name)
        model_path = "/Users/kravisankaran/Desktop/big-data-final-project/cross_val_model_129_Runs"
    else:
        print("Run number : ", file_name)
        model_path = file_name_string
    model = PipelineModel.load(model_path)
    stages_steps = model.stages
    train_shuffle_small = shuffled_train_df.filter( (col('row_num') >= i) & (col('row_num') <=i+90000))
    stem2 = PorterStemming(inputCol="tokens", outputCol="stemmed")
    stem_pipeline = Pipeline(stages= [tk, stem2]).fit(train_shuffle_small)
    train_stem = stem_pipeline.transform(train_shuffle_small)\
                          .where(F.size(F.col("stemmed")) >= 1)
    updated_model = Pipeline(stages = stages_steps).fit(train_stem)
    predictions = updated_model.transform(test_stem)
    score         = evaluator.evaluate(predictions)
    print("AUC SCORE in this run : {}".format(score))
    file_name_string = "cross_val_model_" + str(file_name + 1) + "_Runs"
    updated_model.save(file_name_string)
    print("Model saved with name : ", file_name_string)
    file_name += 1
    i += 90000

Run number :  129


24/04/27 09:29:14 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/27 09:29:15 WARN MemoryStore: Not enough space to cache rdd_296_0 in memory! (computed 451.6 MiB so far)


AUC SCORE in this run : 0.8242746099293129


24/04/27 09:39:51 WARN TaskSetManager: Stage 1179 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 09:39:52 WARN TaskSetManager: Stage 1183 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_130_Runs
Run number :  130


...

Model saved with name :  cross_val_model_136_Runs
Run number :  136


24/04/27 10:07:14 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/27 10:07:19 WARN MemoryStore: Not enough space to cache rdd_296_0 in memory! (computed 2.5 GiB so far)


AUC SCORE in this run : 0.8242746099293129


24/04/27 10:11:50 WARN TaskSetManager: Stage 1480 contains a task of very large size (3196 KiB). The maximum recommended task size is 1000 KiB.
24/04/27 10:11:51 WARN TaskSetManager: Stage 1484 contains a task of very large size (1602 KiB). The maximum recommended task size is 1000 KiB.


Model saved with name :  cross_val_model_137_Runs


In [44]:
predictedAndLabels = predictions.select(["prediction","label"])\
                                .rdd.map(lambda r : (float(r[0]), float(r[1])))
from pyspark.mllib.evaluation import MulticlassMetrics
metrics = MulticlassMetrics(predictedAndLabels)

print("Test Set Accuracy: {}".format(metrics.accuracy))

/Users/kravisankaran/trial/lib/python3.12/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
24/04/27 10:11:52 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/27 10:11:56 WARN MemoryStore: Not enough space to cache rdd_296_0 in memory! (computed 2.5 GiB so far)
24/04/27 10:11:59 WARN DAGScheduler: Broadcasting large task binary with size 4.7 MiB
24/04/27 10:12:03 WARN MemoryStore: Not enough space to cache rdd_296_0 in memory! (computed 2.5 GiB so far)


Test Set Accuracy: 0.857287891533634
